\begin{center}
Gabe Morris
\end{center}

In [1]:
# Notebook Preamble
import sympy as sp
import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use('maroon_ipynb.mplstyle')

def sub_return(expression, substitution, sym=None):
    """
    Displays a substituted expression and returns the value.

    :param expression: Symbolic expression
    :param substitution: A list of tuples to be passed into the .sub method
    :param sym: An optional symbol to display prior to operations
    :return: A simplified value of what is substituted.
    """
    with sp.evaluate(False):
        expr_sub = expression.subs(substitution)
    simp = expr_sub.doit().n()
    if sym is None:
        lev1 = sp.Eq(expression, expr_sub)
        lev2 = sp.Eq(lev1, simp, evaluate=False)
        display(lev2)
    else:
        lev1 = sp.Eq(sym, expression)
        lev2 = sp.Eq(lev1, expr_sub, evaluate=False)
        lev3 = sp.Eq(lev2, simp, evaluate=False)
        display(lev3)
    return simp

def get_principal(s_x, s_y, t_xy):
    """
    Shows the principal stress calculations for a 2D element.

    :param s_x: normal stress along the x-axis
    :param s_y: normal stress along the y-axis
    :param t_xy: shear stress on face x in the direction of y
    :return: principal stresses
    """
    s_x_, s_y_, t_xy_ = sp.symbols(r'\sigma_x \sigma_y \tau_{xy}')
    symbols_ = sp.symbols(r'\sigma_1 \sigma_2 \tau_1 \tau_2')
    expr1_ = (s_x_ + s_y_)/2 + sp.sqrt(((s_x_ - s_y_)/2)**2 + t_xy_**2)
    expr2_ = (s_x_ + s_y_)/2 - sp.sqrt(((s_x_ - s_y_)/2)**2 + t_xy_**2)
    expr3_ = sp.sqrt(((s_x_ - s_y_)/2)**2 + t_xy_**2)
    expr4_ = -sp.sqrt(((s_x_ - s_y_)/2)**2 + t_xy_**2)
    sub_list = [(s_x_, s_x), (s_y_, s_y), (t_xy_, t_xy)]
    ans = []
    for sym_, expr_ in zip(symbols_, [expr1_, expr2_, expr3_, expr4_]):
        ans.append(sub_return(expr_, sub_list, sym=sym_))
    return ans

def von_mises(s_x, s_y, s_z, t_xy, t_yz, t_zx):
    return 1/sp.sqrt(2)*sp.sqrt((s_x - s_y)**2 + (s_y - s_z)**2 + (s_z - s_x)**2 + 6*(t_xy**2 + t_yz**2 + t_zx**2))

def log10(x_):
    return sp.log(x_)/sp.log(10)

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 4403 & Final Exam & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 3-83
## Given

\begin{center}
\includegraphics{images/fig1}
\end{center}

A gear reduction unit uses the countershaft shown in the figure above. Gear A receives power from another gear with a transmitted force $F_A$ applied at the $20^\circ$ pressure angle as shown. The power is transmitted through the shaft and delivered through gear B through a transmitted force $F_B$ at the pressure angle shown.

## Find
a. Determine the force $F_B$, assuming the shaft is running at a constant speed.
b. Find the bearing reaction forces, assuming the bearings act as simple supports.
c. Draw a shear-force and bending moment diagrams for the shaft. If needed, make one set for the horizontal plane and another set for the vertical plane.
d. At the point of maximum bending moment, determine the bending stress and the torsional shear stress.
e. At the point of maximum bending moment, determine the principal stresses and the maximum shear stress.

## Solution
### Part A and Part B
$F_B$ and the reaction forces may be found by summing the forces and taking the moments about the point $O$.

In [2]:
# Using matrices takes less thinking and more letting python do the work.
Oy, Oz, Cy, Cz, F_B = sp.symbols('O_y O_z C_y C_z F_B')
zero = sp.ZeroMatrix(3, 1)
F_A_mat = sp.Matrix([0, 300*sp.cos(sp.rad(20)), -300*sp.sin(sp.rad(20))])
F_B_mat = sp.Matrix([0, -F_B*sp.sin(sp.rad(20)), F_B*sp.cos(sp.rad(20))])
O = sp.Matrix([0, Oy, Oz])
C = sp.Matrix([0, Cy, Cz])

# Sum forces
sys1 = sp.Eq(O + F_A_mat + F_B_mat + C, zero)

# Sum moments about O
rA = sp.Matrix([16, 0, 10])
rC = sp.Matrix([16 + 14, 0, 0])
rB = sp.Matrix([16 + 14 + 9, 4, 0])
sys2 = sp.Eq(rA.cross(F_A_mat) + rC.cross(C) + rB.cross(F_B_mat), zero)

sol = sp.solve([sys1, sys2], dict=True)[0]
for key, value in sol.items():
    display(sp.Eq(key, value.n()))  # lbf

Eq(C_y, 183.118820416782)

Eq(C_z, -861.477082334154)

Eq(F_B, 750.0)

Eq(O_y, -208.511499158303)

Eq(O_z, 259.313659742423)

The reaction forces above assume all positive directions.

### Part C

\begin{center}
\includegraphics{images/fig2}
\end{center}

### Part D
The magnitudes of the moment at section A and section C must be compared.

In [3]:
M_A = sp.sqrt(sp.S('3336.18399')**2 + sp.S('4149.01856')**2)
M_C = sp.sqrt(sp.S('6342.92519')**2 + sp.S('2308.63597')**2)
display(M_A, M_C)

5323.95328927262

6750.00000058625

In [4]:
# M_C is the greater value
M_C = sp.S(6750)  # in*lbf
d = sp.S('1.25')
sig = M_C*32/(sp.pi*d**3)  # psi
T = 300*sp.cos(sp.rad(20))*10  # static torque
tau = T*16/(sp.pi*d**3)  # psi
display(sig.n(), tau.n())

35202.5269328378

7351.01217595663

### Part E

In [5]:
_ = get_principal(sig.n(5), 0, tau.n(5))

Eq(Eq(Eq(\sigma_1, \sigma_x/2 + \sigma_y/2 + sqrt(\tau_{xy}**2 + (\sigma_x/2 - \sigma_y/2)**2)), (1/2)*0 + (1/2)*35203.0 + sqrt(7351.0**2 + (-1/2*0 + (1/2)*35203.0)**2)), 36675.875)

Eq(Eq(Eq(\sigma_2, \sigma_x/2 + \sigma_y/2 - sqrt(\tau_{xy}**2 + (\sigma_x/2 - \sigma_y/2)**2)), -sqrt(7351.0**2 + (-1/2*0 + (1/2)*35203.0)**2) + (1/2)*0 + (1/2)*35203.0), -1473.375)

Eq(Eq(Eq(\tau_1, sqrt(\tau_{xy}**2 + (\sigma_x/2 - \sigma_y/2)**2)), sqrt(7351.0**2 + (-1/2*0 + (1/2)*35203.0)**2)), 19074.625)

Eq(Eq(Eq(\tau_2, -sqrt(\tau_{xy}**2 + (\sigma_x/2 - \sigma_y/2)**2)), -sqrt(7351.0**2 + (-1/2*0 + (1/2)*35203.0)**2)), -19074.625)

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Final Exam & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 5-36
## Given

\begin{center}
\includegraphics{images/fig3}
\end{center}

The figure above illustrates that the factor of safety for a machine element depends on the particular point selected for analysis. The bar is made of AISI 1006 cold-drawn steel and is loaded by the forces $F=0.55\,kN$, $P=4.0\,kN$, and $T=25\,N\cdot m$.

## Find
Find the factor of safety for both elements using the distortion energy method.

## Solution
The yield strength is $S_y=280\,MPa$.

### Element A

In [6]:
P = sp.S(4_000)
Sy = sp.S(280e6)
T = sp.S(25)
d = sp.S('0.015')
F = sp.S('0.55e3')
l = sp.S('0.1')

sig_x = 4*P/(sp.pi*d**2)
tau_xy = 16*T/(sp.pi*d**3) - 4*sp.S('0.55e3')/(3*(sp.pi/4)*(sp.S('0.015')**2))
display(sig_x.n(), tau_xy.n())  # Pa

22635369.6841807

33575798.3648680

In [7]:
sig_prime = von_mises(sig_x, 0, 0, tau_xy, 0, 0)
sig_prime.n()  # Pa

62404828.8857068

In [8]:
(Sy/sig_prime).n()

4.48683226922735

### Element B

In [9]:
sig_x = 32*F*l/(sp.pi*d**3) + 4*P/(sp.pi*d**2)
tau_xy = 16*T/(sp.pi*d**3)
display(sig_x.n(), tau_xy.n())  # Pa

188628080.701506

37725616.1403011

In [10]:
sig_prime = von_mises(sig_x, 0, 0, tau_xy, 0, 0)
sig_prime.n()  # Pa

199625196.727842

In [11]:
(Sy/sig_prime).n()

1.40262854884865

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Final Exam & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-62
## Given
The material properties of a machine part are $S_{ut}=85\,ksi$, $f=0.86$, and a fully corrected endurance limit of $S_e=45\,ksi$. The part is to be cycled at $\sigma_a=35\,ksi$ and $\sigma_m=30\,ksi$ for $12\cdot 10^3$ cycles.

## Find
Using the Goodman criterion, estimate the new endurance limit after cycling.

a. Use Miner's method.
b. Use Manson's method.

## Solution
### Part A

In [12]:
Se = sp.S(45)
Sut = sp.S(85)
sig_a = sp.S(35)
sig_m = sp.S(30)
N = sp.S(12_000)
# f = sp.S('1.06') - sp.S('2.8e-3')*Sut + sp.S('6.9e-6')*Sut**2
f = sp.S('0.86')

sig_ar = sig_a/(1 - (sig_m/Sut))  # 6-59
sig_ar.n()  # ksi

54.0909090909091

In [13]:
a = (f*Sut)**2/Se
b = -sp.Rational(1, 3)*log10(f*Sut/Se)
display(a, b.n())

118.746888888889

-0.0702349543941723

The total number of cycles needs to be calculated, then the remaining number of cycles may be found.

In [14]:
N1 = (sig_ar/a)**(1/b)
N1.n()

72815.2378713139

In [15]:
N_remaining = N1 - N
N_remaining.n()

60815.2378713139

In [16]:
a_prime = sig_ar/(N_remaining**b)
Se_prime = a_prime*1e6**b
Se_prime.n()  # ksi

44.4344130331323

### Part B
See p. 355.

In [17]:
Sf = f*Sut
b_prime = log10(Sf/sig_ar)/log10(1e3/N_remaining)
a_prime = sig_ar/(N_remaining**b_prime)
Se_prime = a_prime*1e6**b_prime
Se_prime.n()  # ksi

44.0529868187135

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Final Exam & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 8-26
## Given
A $\frac{3}{4}$ in-16 UNF series SAE grade 5 bolt has a $\frac{3}{4}$-in ID steel tube 10 in long, clamped between washer faces of bolt and nut by turning the nut snug and adding one-third of a turn. The tube OD is the washer face diameter $d_w=1.5d=1.5(0.75)=1.125$ in.

## Find
a. Determine the bolt stiffness, the tube stiffness, and the joint constant $C$.
b. When the one-third turn of the nut is applied, what is the initial tension $F_i$ in the bolt?

## Solution
### Part A
The bolt stiffness comes from Eq. 8-17.

In [18]:
l = 10  # inches
H = sp.Rational(41, 64)
L = l + H
L.n()

10.6406250000000

In [19]:
d = sp.S('0.75')
L = sp.S('10.75')  # to the nearest quarter inch
LT = 2*d + sp.S('0.5')
ld = L - LT
lt = l - ld
Ad = sp.pi*d**2/4
At = sp.S('0.373')  # table 8-2
E = sp.S(30e6)  # psi (Table A-5)
kb = Ad*At*E/(Ad*lt + At*ld)
kb.n()  # psi

1295495.93286476

The stiffness the member may be calculated as a long bar (Eq. 4-4),

$$k=\frac{AE}{l}$$

In [20]:
km = sp.pi/4*(sp.S('1.125')**2 - sp.S('0.75')**2)*E/l
km.n()

1656699.25091649

On p. 448, the joint constant is

$$C=\frac{k_b}{k_b+k_m}$$

In [21]:
C = kb/(kb + km)
C.n()

0.438824621075850

### Part B
The total displacement may be described as, 

$$\delta=\delta_b+\delta_m=\frac{N_{turns}}{N_{threads}}$$

In [22]:
Fi, kb_, km_, Nt_, N = sp.symbols('F_i k_b k_m N_t N')
db = Fi/kb_
dm = Fi/km_
eq = sp.Eq(db + dm, Nt_/N)
sol = sp.solve(eq, Fi)[0]
_ = sub_return(sol, [(km_, km.n(5)), (kb_, kb.n(5)), (Nt_, sp.Rational(1, 3)), (N, 16)], sym=Fi)  # lbf

Eq(Eq(Eq(F_i, N_t*k_b*k_m/(N*(k_b + k_m))), (1/3)*1.2955e+6*1.6567e+6/(16*(1.2955e+6 + 1.6567e+6))), 15145.84375)